In [ ]:
import numpy as np

img = np.random.randint(0, 50, (1, 1, 7, 7)) # 任意の範囲の整数の乱数、最小値0、最大値50
# img = np.round(img)
print(img.shape)

print(img[0].shape)
print(img[0])

In [3]:
# 画像データの持ち方の確認
import numpy as np

A = np.array(
    [[["000", "001", "002", "003"],
      ["010", "011", "012", "013"],
      ["020", "021", "022", "023"]],
     [["100", "101", "102", "103"],
      ["110", "111", "112", "113"],
      ["120", "121", "122", "123"]]]
)
print("----------------A---------------")
print(A)
print("---------------A[0]--------------")
print(A[0])
print("---------------A[0, 1]------------")
print(A[0, 1])
print("---------------A[0, 1, 3]---------")
print(A[0, 1, 3])
print("---------------A.shape------------")
print(A.shape)

----------------A---------------
[[['000' '001' '002' '003']
  ['010' '011' '012' '013']
  ['020' '021' '022' '023']]

 [['100' '101' '102' '103']
  ['110' '111' '112' '113']
  ['120' '121' '122' '123']]]
---------------A[0]--------------
[['000' '001' '002' '003']
 ['010' '011' '012' '013']
 ['020' '021' '022' '023']]
---------------A[0, 1]------------
['010' '011' '012' '013']
---------------A[0, 1, 3]---------
013
---------------A.shape------------
(2, 3, 4)


In [4]:

# reshapeの動作確認
B = A.reshape(2 * 3 * 4)
# B = A.reshape(-1)と同じ意味。
print(B)

['000' '001' '002' '003' '010' '011' '012' '013' '020' '021' '022' '023'
 '100' '101' '102' '103' '110' '111' '112' '113' '120' '121' '122' '123']


In [8]:
# 引数
# img: 画像データ
# k_h: カーネル高さ
# k_w: カーネル幅
# s_h: ストライド幅
# s_w: ストライド高さ
# p_h: パディング高さ
# p_w: パディング幅
# ストライド量、パディング量は縦横まとめられる場合あり
def im2col(img, k_h, k_w, s_h, s_w, p_h, p_w):
    """
    画像を「フィルターを適用する場所」ごとに切り出し、横一行に並べ替える関数
    (4次元の画像データを2次元の行列に変換する)
    """
    # 1. 元の画像のサイズ（データ数、色数、高さ、幅）を取得
    # - n: データ数
    # - c: チャンネル数
    # - h: 画像の高さ
    # - w: 画像の幅
    n, c, h, w = img.shape
    print(img.shape)

    # 2. パディング処理（画像の周りに0の「縁取り」を作る）
    # 高さ方向(p_h)と幅方向(p_w)に0を追加。バッチ数とチャンネル数には追加しない。
    img = np.pad(img, [(0,0), (0,0), (p_h, p_h), (p_w, p_w)], 'constant')
    print(img[0])
    print(img.shape)

    # 3. 畳み込み後の「出力サイズ」を計算
    # 公式: (元のサイズ + 2×パディング - フィルターサイズ) // ストライド + 1
    out_h = (h + 2*p_h - k_h)//s_h + 1
    out_w = (w + 2*p_w - k_w)//s_w + 1

    # 4. データを一時的に格納する「6次元の箱」を準備
    # 形状: (データ数, チャンネル数, フィルター高さ, フィルター幅, 出力高さ, 出力幅)
    col = np.ndarray((n, c, k_h, k_w, out_h, out_w), dtype=img.dtype)  # 戻り値となる4次元配列を準備。(データ数、チャンネル数、カーネル高さ、カーネル幅、出力高さ、出力幅)
    print(col.shape)
    print(col[0])

    # 5. 【重要】フィルター内の各ピクセル位置(y, x)ごとに、画像から一括抽出
    for y in range(k_h):
        # y_lim：最後のフィルターの位置
        y_lim = y + s_h * out_h

        for x in range(k_w):
            # スライスする幅の終点を計算
            x_lim = x + s_w * out_w

            # 画像全体から「ストライド間隔」で画素を抜き出し、colの該当位置に流し込む
            # これにより「1つずつ切り出す」ループを避けて高速化している
            col[:, :, y, x, :, :] = img[:, :, y:y_lim:s_h, x:x_lim:s_w]

    # 6. 並び替えと整形（ここが行列計算への橋渡し）
    # 軸を (データ数, 出力高, 出力幅, チャンネル, フィルタ高, フィルタ幅) の順に入れ替え
    # 軸の順番を変更して、reshapeしやすいようにする
    col = col.transpose(0, 4, 5, 1, 2, 3)
    
    # 行: 「1つのパッチ(切り抜き)」を横1行に並べる
    # 列: データの総数(n * out_h * out_w)
    # これで (パッチ数, パッチ内の全データ) という2次元行列になる
    col = col.reshape(n*out_h*out_w, -1)
    return col